# Matrix product states with mpnum: Introduction

mpnum implements *matrix product arrays* (MPA), which are efficient parametrisations of certain multi-partite arrays. Examples are *matrix product states* (MPS) and *matrix product operators* (MPO) with one and two array indices per site, respectively. The main class implementing a MPA with arbitrary number of array indices (or "physical legs") is ``mpnum.MPArray``. 

In [1]:
import numpy as np
from numpy import linalg as la

import mpnum as mp

## MPA and MPS basics

A convenient example to deal with is a random MPA. First, we create a fixed seed, then a random MPA:

In [2]:
rng = np.random.RandomState(seed=42)
mpa = mp.random_mpa(sites=4, ldim=2, bdim=3, randstate=rng, normalized=True)

The MPA is an instance of the MPArray class:

In [3]:
mpa

Number of sites:

In [4]:
len(mpa)

4

Number of physical legs:

In [5]:
mpa.plegs

(1, 1, 1, 1)

Indeed, we have created a matrix product *state* with one physical leg per site.

Dimension of each physical leg:

In [6]:
mpa.pdims

((2,), (2,), (2,), (2,))

Note that the number and dimension of the physical legs at each site can differ (altough this is rarely used in practice). 

Bond dimension between each pair of sites (dummy bonds before and after the chain are omitted, currently mpnum only implements open boundary conditions):

In [7]:
mpa.bdims

(2, 3, 2)

Above, we have specified ``normalized=True``.  Therefore, we have an MPA with $\ell_2$-norm 1.  In case the MPA does not represent a vector, but has more physical legs, it is nonetheless treated as a vector. Hence, for operators <code>mp.norm</code> implements the Frobenius norm.

In [8]:
mp.norm(mpa)

0.99999999999999989

Convert to full array:

In [9]:
arr = mpa.to_array()
arr.shape

(2, 2, 2, 2)

The resulting full array has one index for each physical leg.

Now convert the full array back to an MPA:

In [10]:
mpa2 = mp.MPArray.from_array(arr)
len(mpa2)

1

We have obtained an MPA with length 1.  This is not what we expected.  The reason is that by default, all physical legs are placed on a single site:  (Also notice the difference between ``mpa2.pdims`` here and ``mpa.pdims`` above.)

In [11]:
mpa2.plegs, mpa2.pdims

((4,), ((2, 2, 2, 2),))

We obtain the desired result by specifying the number of plegs per site we want:

In [12]:
mpa2 = mp.MPArray.from_array(arr, plegs=1)
len(mpa2)

4

Finally, we can compute the norm distance between the two MPAs.  (Again, the Frobenius norm is used.)

In [13]:
mp.norm(mpa - mpa2)

9.6600784925655173e-16

Since this is an often used operation and allows for additional optimization (not implemented currently), it is advisable to use the specific <code>mp.normdist</code> for this:

In [14]:
mp.normdist(mpa, mpa2)

9.6600784925655173e-16

Sums, differences and scalar multiplication of MPAs is done with the normal operators:

In [15]:
mp.norm(3 * mpa)

2.9999999999999996

In [16]:
mp.norm(mpa + 0.5 * mpa)

1.5000000000000002

In [17]:
mp.norm(mpa - 1.5 * mpa)

0.50000000000000067

Addition (and subtraction) double the bond dimension, while scalar multiplication leaves the bond dimension unchanged:

In [18]:
mpa.bdims

(2, 3, 2)

In [19]:
(3 * mpa).bdims

(2, 3, 2)

Note that the bond dimension of ``mpa`` has changed.  This is a side effect of ``mp.norm(mpa)`` above, which calls ``mpa.normalize()``. This transforms ``mpa`` to a suitable normal form and may reduce bond dimensions without loss of information.

In [20]:
mpa2 = mp.random_mpa(sites=4, ldim=2, bdim=2, randstate=rng)
mpa2.bdims

(2, 2, 2)

In [21]:
(mpa + mpa2).bdims

(4, 5, 4)

## MPO basics

First, we create a random MPA with two physical legs per site:

In [22]:
mpo = mp.random_mpa(sites=4, ldim=(3, 2), bdim=3, randstate=rng, normalized=True)

Basic properties:

In [23]:
len(mpo), mpo.plegs, mpo.bdims

(4, (2, 2, 2, 2), (3, 3, 3))

Each site has two physical legs, one with dimension 3 and one with dimension 2.  This corresponds to a non-square full array.

In [24]:
mpo.pdims

((3, 2), (3, 2), (3, 2), (3, 2))

Now convert the mpo to a full array:

In [25]:
mpo_arr = mpo.to_array()
mpo_arr.shape

(3, 2, 3, 2, 3, 2, 3, 2)

We refer to this arangement of axes as _local form_, since indices, which correspond to the same site, are neighboring. This is a natural form for the MPO representation. However, for some operations it is more convenient to have row and column indices grouped together -- we refer to this as _global form_:

In [26]:
from mpnum._tools import local_to_global

mpo_arr = mpo.to_array()
mpo_arr = local_to_global(mpo_arr, sites=len(mpo))
mpo_arr.shape

(3, 3, 3, 3, 2, 2, 2, 2)

This gives the expected result.  Note that it is crucial to specify the correct number of sites, otherwise we do not get what we want:

In [27]:
mpo_arr = mpo.to_array()
mpo_arr = local_to_global(mpo_arr, sites=2)
mpo_arr.shape

(3, 3, 2, 2, 3, 3, 2, 2)

As an alternative, there is the following shorthand:

In [28]:
mpo_arr = mpo.to_array_global()
mpo_arr.shape

(3, 3, 3, 3, 2, 2, 2, 2)

## MPO-MPS product and arbitrary MPA-MPA products

We can now compute the matrix-vector product of ``mpa`` from above (which is an MPS) and ``mpo``.

In [29]:
prod = mp.dot(mpo, mpa, axes=(-1, 0))
prod.pdims

((3,), (3,), (3,), (3,))

The result is a new MPS, with local dimension changed by ``mpo``.  The ``axes`` argument is optional and defaults to ``axes=(-1, 0)`` -- i.e. contracting the last pyhsical index of the first factor with the first physical index of the second factor.  More specifically, the ``axes`` argument  specifies which physical legs should be contracted:  ``axes[0]`` specifies the physical in the first argument, and ``axes[1]`` specifies the physical leg in the second argument.  This means that the same product can be achieved with

In [30]:
prod2 = mp.dot(mpa, mpo, axes=(0, 1))
mp.normdist(prod, prod2)

1.1908430499814518e-16

Note that in any case, the bond dimensions of the output of ``mp.dot`` are the *product* of the original bond dimension:

In [31]:
mpo.bdims, mpa.bdims, prod.bdims

((3, 3, 3), (2, 3, 2), (6, 9, 6))

Now we compute the same product using the full arrays ``arr`` and ``mpo_arr``:

In [32]:
arr_vec = arr.reshape(16)
mpo_arr_matrix = mpo_arr.reshape((81, 16))
prod3_vec = np.dot(mpo_arr_matrix, arr_vec)
prod3_vec.shape

(81,)

As you can see, we need to reshape the result ``prod3_vec`` before we can convert it back to an MPA:

In [33]:
prod3_arr = prod3_vec.reshape((3, 3, 3, 3))
prod3 = mp.MPArray.from_array(prod3_arr, plegs=1)
prod3.pdims

((3,), (3,), (3,), (3,))

Now we can compare the two results:

In [34]:
mp.normdist(prod, prod3)

1.6857435839227793e-16

We can also compare by converting ``prod`` to a full array:

In [35]:
prod_arr = prod.to_array()
la.norm((prod3_arr - prod_arr).reshape(81))

8.0813145555190655e-17

## Converting full operators to MPOs

While MPO algorithms avoid using full operators in general, we will need to convert a term acting on only two sites to an MPO in order to continue with MPO operations;  i.e. we will need to convert a full array to an MPO. 

First, we define a full operator:

In [36]:
CZ = np.array([[ 1.,  0.,  0.,  0.],
               [ 0.,  1.,  0.,  0.],
               [ 0.,  0.,  1.,  0.],
               [ 0.,  0.,  0., -1.]])

This operator is the so-called *controlled Z* gate: Apply ``Z`` on the second qubit if the first qubit is in state ``e2``.

To convert it to an MPO, we have to reshape:

In [37]:
CZ_arr = CZ.reshape((2, 2, 2, 2))

Now we can create an MPO, being careful to specify the correct number of legs per site:

In [38]:
CZ_mpo = mp.MPArray.from_array_global(CZ_arr, plegs=2)

To test it, we apply the operator to the state which has both qubits in state ``e2``:

In [39]:
vec = np.kron([0, 1], [0, 1])
vec

array([0, 0, 0, 1])

Reshape and convert to an MPS:

In [40]:
vec_arr = vec.reshape((2, 2))
mps = mp.MPArray.from_array(vec_arr, plegs=1)

Now we can compute the matrix-vector product:

In [41]:
out = mp.dot(CZ_mpo, mps)
out.to_array().ravel()

array([ 0.,  0.,  0., -1.])

The output is as expected:  We have acquired a minus sign. 

We have to be careful to use ``from_array_global`` and not ``from_array``, because the ``CZ_arr`` is in *global form*. Here, all physical legs have the same dimension, so we can use ``from_array`` without error:

In [42]:
CZ_mpo2 = mp.MPArray.from_array(CZ_arr, plegs=2)

However, the result is not what we want:

In [43]:
out2 = mp.dot(CZ_mpo2, mps)
out2.to_array().ravel()

array([ 1.,  0.,  0., -1.])

The reason is easy to see: We have applied the following matrix to our state:

In [44]:
CZ_mpo2.to_array_global().reshape(4, 4)

array([[ 1.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 1.,  0.,  0., -1.]])

Keep in mind that we have to use ``to_array_global`` before the reshape.  Using ``to_array`` would not provide us the matrix which we have applied to the state with ``mp.dot``.  Instead, it will exactly return the input:

In [45]:
CZ_mpo2.to_array().reshape(4, 4)

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0., -1.]])

Again, ``from_array_global`` is just the shorthand for the following:

In [46]:
from mpnum._tools import global_to_local

CZ_mpo3 = mp.MPArray.from_array(global_to_local(CZ_arr, sites=2), plegs=2)

mp.normdist(CZ_mpo, CZ_mpo3)

1.424948699751694e-16

As you can see, in the explicit version you must submit both the correct number of sites and the correct number of physical legs per site.

## Creating MPAs from Kronecker products

It is a frequent task to create an MPS which represents the product state of $\vert 0 \rangle$ on each qubit.  If the chain is very long, we cannot create the full array with ``np.kron`` and use ``MPArray.from_array`` afterwards because the array would be too large. 

First, we need the state on a single site:

In [47]:
e1 = np.array([1, 0])
e1

array([1, 0])

Then we can use ``from_kron`` to directly create an MPS representation of the Kronecker product:

In [48]:
mps = mp.MPArray.from_kron([e1, e1, e1])
mps.to_array().ravel()

array([1, 0, 0, 0, 0, 0, 0, 0])

This works well for large numbers of sites because the needed memory scales linearly with the number of sites:

In [49]:
mps = mp.MPArray.from_kron([e1] * 200)
len(mps)

200

Do not call ``.to_array()`` on this state!

The bond dimension of the state is 1, because it is a product state:

In [50]:
np.array(mps.bdims)  # Convert to an array for nicer display

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

We can also create a single-site MPS:

In [51]:
mps1 = mp.MPArray.from_array(e1, plegs=1)
len(mps1)

1

After that, we can use mp.outer to create Kronecker products of the MPS directly:

In [52]:
mps = mp.outer([mps1, mps1, mps1])
len(mps)

3

It returns the same result as before:

In [53]:
mps.to_array().ravel()

array([1, 0, 0, 0, 0, 0, 0, 0])

We can also use ``mp.outer`` on the three-site MPS:

In [54]:
mps = mp.outer([mps] * 100)
len(mps)

300

## Compression

A typical matrix product based numerical algorithm performs many additions or multiplications of MPAs.  As mentioned above, both operations increase the bond dimension.  If we let the bond dimension grow, the amount of memory we need grows with the number of operations we perform.  To avoid this problem, we have to find an MPA with a smaller bond dimension which is a good approximation to the original MPA.

We start by creating an MPO representation of the identity matrix on 6 sites with local dimension 3:

In [55]:
op = mp.eye(sites=6, ldim=3)

In [56]:
op.pdims

((3, 3), (3, 3), (3, 3), (3, 3), (3, 3), (3, 3))

As it is a tensor product operator, it has bond dimension 1:

In [57]:
op.bdims

(1, 1, 1, 1, 1)

However, addition adds bond dimensions:

In [58]:
op2 = op + op + op
op2.bdims

(3, 3, 3, 3, 3)

Matrix multiplication multiplies bond dimensions:

In [59]:
op3 = mp.dot(op2, op2)
op3.bdims

(9, 9, 9, 9, 9)

(NB: .compress or .compression below can call .normalize on op3, which in turn could already reduce the bond dimension to 1.  Keep that in mind.)

Keep in mind that the operator represented by ``op3`` is still the identity operator, i.e. a tensor product operator.  This means that we expect to find a good approximation with low bond dimension easily.  Finding such an approximation is called *compression* and is achieved as follows:

In [60]:
copy = op3.copy()
overlap = copy.compress(method='svd', bdim=1)
overlap, copy.bdims

(59048.999999999935, (1, 1, 1, 1, 1))

Calling ``.compress`` on an MPA replaces the MPA in place with a version with smaller bond dimension.  Overlap gives the absolute value of the (Hilbert-Schmidt) inner product between the original state and the output:

Instead of in-place compression, we can also obtain a compressed copy:

In [61]:
compr, overlap = op3.compression(method='svd', bdim=2)
overlap, compr.bdims, op3.bdims

(59048.999999999935, (1, 1, 1, 1, 1), (9, 9, 9, 9, 9))

Here, SVD compression has returned a compression with bond dimension 1, even though we requested (maximal) bond dimension 2.

SVD compression can also be told to meet a certain truncation error threshold per site (see the documentation of ``mp.MPArray.compress`` for details.

In [62]:
compr, overlap = op3.compression(method='svd', relerr=1e-6)
overlap, compr.bdims, op3.bdims

(59048.999999999935, (1, 1, 1, 1, 1), (9, 9, 9, 9, 9))

We can also use variational compression instead of SVD compression:

In [63]:
compr, overlap = op3.compression(method='var', bdim=2, num_sweeps=10, var_sites=2)
# Convert overlap from numpy array with shape () to float for nicer display:
overlap = overlap.reshape((1,))[0]
complex(overlap), compr.bdims, op3.bdims

((59048.999999999905+0j), (2, 2, 2, 2, 2), (9, 9, 9, 9, 9))

In this case, we have requested and obtained a compression with maximal bond dimension 2.

As a reminder, it is always advisable to check whether the overlap between the input state and the compression is large enough.  In a involved algorithm, it can be useful to store the compression error at each invocation of compression.

## MPO sum of local terms

A frequent task is to compute the MPO representation of a local Hamiltonian, i.e. of an operator of the form
$$ H = \sum_{i=1}^{n-1} h_{i, i+1} $$
where $h_{i, i+1}$ acts only on sites $i$ and $i + 1$.  This means that $h_{i, i+1} = \mathbb 1_{i - 1} \otimes h'_{i, i+1} \otimes \mathbb 1_{n - w + 1}$ where $\mathbb 1_k$ is the identity matrix on $k$ sites and $w = 2$ is the width of $h'_{i, i+1}$.

We show how to obtain an MPO representation of such a Hamiltonian.  First of all, we need to define the local terms.  For simplicity, we choose $h'_{i, i+1} = \sigma_Z \otimes \sigma_Z$ independently of $i$.

In [64]:
zeros = np.zeros((2, 2))
zeros

array([[ 0.,  0.],
       [ 0.,  0.]])

In [65]:
idm = np.eye(2)
idm

array([[ 1.,  0.],
       [ 0.,  1.]])

In [66]:
# Create a float array instead of an int array to avoid problems later
Z = np.diag([1., -1])
Z

array([[ 1.,  0.],
       [ 0., -1.]])

In [67]:
h = np.kron(Z, Z)
h

array([[ 1.,  0.,  0.,  0.],
       [ 0., -1.,  0., -0.],
       [ 0.,  0., -1., -0.],
       [ 0., -0., -0.,  1.]])

First, we have to convert the local term ``h`` to an MPO:

In [68]:
h_arr = h.reshape((2, 2, 2, 2))
h_mpo = mp.MPArray.from_array_global(h_arr, plegs=2)
h_mpo.bdims

(4,)

h_mpo has bond dimension 4 even though h is a tensor product.  This is far from optimal.  We improve things as follows:  (We could also compress ``h_mpo``.)

In [69]:
h_mpo = mp.MPArray.from_kron([Z, Z])
h_mpo.bdims

(1,)

The most simple way is to implement the formula from above with MPOs:  First we compute the $h_{i, i+1}$ from the $h'_{i, i+1}$:

In [75]:
width = 2
sites = 6
local_terms = []

for startpos in range(sites - width + 1):
    left = [mp.MPArray.from_kron([idm] * startpos)] if startpos > 0 else []
    right = [mp.MPArray.from_kron([idm] * (sites - width - startpos))] \
        if sites - width - startpos > 0 else []
    h_at_startpos = mp.outer(left + [h_mpo] + right)
    local_terms.append(h_at_startpos)

local_terms

Next, we compute the sum of all the local terms and check the bond dimension of the result:

In [76]:
H = local_terms[0]

for local_term in local_terms[1:]:
    H += local_term
    
H.bdims

(5, 5, 5, 5, 5)

The bond dimension is explained by the bond dimension of the local terms:

In [77]:
[local_term.bdims for local_term in local_terms]

[(1, 1, 1, 1, 1),
 (1, 1, 1, 1, 1),
 (1, 1, 1, 1, 1),
 (1, 1, 1, 1, 1),
 (1, 1, 1, 1, 1)]

We just have to add the bond dimensions at each position.

``mpnum`` provides a function which constructs ``H`` from ``h_mpo``, with a output MPO with smaller bond dimension by taking into account the trivial action on some sites:

In [78]:
H2 = mp.local_sum([h_mpo] * (sites - width + 1))
H2.bdims

(2, 3, 3, 3, 2)

Without additional arguments, ``mp.local_sum()`` just adds the local terms with the first term starting on site 0, the second on site 1 and so on.  In addition, the length of the chain is chosen such that the last site of the chain coincides with the last site of the last local term.  Other constructions can be obtained by prodividing additional arguments.

We can check that the two Hamiltonians are equal:

In [79]:
mp.normdist(H, H2)

5.7186634519139989e-15

Of course, this means that we could just compress ``H``:

In [80]:
H_comp, overlap = H.compression(method='svd', bdim=3)
overlap

319.99999999999972

In [81]:
H_comp.bdims

(2, 3, 3, 3, 2)

We can also check the minimal bond dimension which can be achieved with SVD compression with small error:

In [82]:
H_comp, overlap = H.compression(method='svd', relerr=1e-6)
overlap

319.99999999999972

In [83]:
H_comp.bdims

(2, 3, 3, 3, 2)

## MPS, MPOs and PMPS

We can represent vectors (e.g. pure quantum states) as MPS, we can represent arbitrary matrices as MPO and we can represent positive semidefinite matrices as purifying matrix product states (PMPS).  For mixed quantum states, we can thus choose between the MPO and PMPS representations.

As mentioned in the introduction, MPS and MPOs are handled as MPAs with one and two physical legs per site.  In addition, PMPS are handled as MPAs with two physical legs per site, where the first leg is the "system" site and the second leg is the corresponding "ancilla" site.

From MPS and PMPS representations, we can easily obtain MPO representations.  ``mpnum`` provides routines for this:

In [84]:
mps = mp.random_mpa(sites=5, ldim=2, bdim=3, normalized=True)
mps_mpo = mp.mps_to_mpo(mps)
mps_mpo.bdims

(4, 9, 9, 4)

As expected, the bond dimension of ``mps_mpo`` is the square of the bond dimension of ``mps``.

Now we create a PMPS with system site dimension 2 and ancilla site dimension 3:

In [85]:
pmps = mp.random_mpa(sites=5, ldim=(2, 3), bdim=3, normalized=True)
pmps.pdims

((2, 3), (2, 3), (2, 3), (2, 3), (2, 3))

In [86]:
pmps_mpo = mp.pmps_to_mpo(pmps)
pmps_mpo.bdims

(9, 9, 9, 9)

Again, the bond dimension is squared, as expected.  We can verify that the first physical leg of each site of ``pmps`` is indeed the system site by checking the physical leg dimension of ``pmps_mpo``:

In [87]:
pmps_mpo.pdims

((2, 2), (2, 2), (2, 2), (2, 2), (2, 2))

## Local reduced states

For state tomography applications, we frequently need the local reduced states of an MPS, MPO or PMPS. We provide the following functions for this task:

* ``mp.reductions_mps_as_pmps()``: Input: MPS, output: local reductions as PMPS
* ``mp.reductions_mps_as_mpo()``: Input: MPS, output: local reductions as MPO
* ``mp.reductions_pmps()``: Input: PMPS, output: Local reductions as PMPS
* ``mp.reductions_mpo()``: Input: MPO, output: Local reductions as MPO

The arguments of all functions are similar, e.g.:

In [88]:
width = 3
startsites = range(len(pmps) - width + 1)
for startsite, red in zip(startsites, mp.reductions_pmps(pmps, width, startsites)):
    print('Reduction starting on site', startsite)
    print('bdims:', red.bdims)
    red_mpo = mp.pmps_to_mpo(red)
    print('trace:', mp.trace(red_mpo))
    print()

Reduction starting on site 0
bdims: (3, 3)
trace: (1-2.30163282106e-18j)

Reduction starting on site 1
bdims: (3, 3)
trace: (1+1.02999206386e-18j)

Reduction starting on site 2
bdims: (3, 3)
trace: (1-8.67361737988e-19j)



Because ``pmps`` was a normalized state, the trace of the reduced states is close to 1.

You can omit the ``startsites`` argument:  The default behaviour is the first reductions starting on site 0, the second on site 1, and so on (which is just what we have requested).  The functions for reduced states can also compute different constructions by providing different arguments not described here. 